In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import exp, pi, cos, sin, atan, sqrt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, callback, Output, Input
from dash.dependencies import Input, Output


In [71]:
app = Dash(__name__)

detuning = np.linspace(-pi, pi, 1000)
detuning_long = np.linspace(-pi, 3*pi, 1000)

def phase_shift(r, a, d):
    return pi + d + atan(r*sin(d)/(a-r*cos(d))) + atan(r*a*sin(d)/(1 - r*a*cos(d)))

phase_shift = np.vectorize(phase_shift)

def transmission_intensity(r, a, d):
    return ((a**2 - 2*r*a*cos(d)+r**2)/(1-2*a*r*cos(d)+(r*a)**2))

transmission_intensity = np.vectorize(transmission_intensity)

def transmission_pass(r1, r2, a, d):
    return (
        (((r2**2)*(a**2))-(2*r1*r2*a*cos(d))+(r1**2))/(1-(2*r1*r2*a*cos(d))+(r1*r2*a)**2)
    )

transmission_pass = np.vectorize(transmission_pass)

def transmission_drop(r1, r2, a, d):
    return (
    ((1-(r1**2))*(1-(r2**2))*a)/(1-(2*r1*r2*a*cos(d))+(r1*r2*a)**2)
    )

def finesse_allpass(r, a):
    return (
    (pi*sqrt(r*a))/(1-(r*a))
    )

def finesse_add_drop(r1, r2, a):
    return(
    (pi*sqrt(r1*r2*a))/(1-(r1**r2*a))
    )

transmission_drop = np.vectorize(transmission_drop)


app.layout = html.Div([
    html.P(children='Self-Coupling Coefficient (r)', style={'textAlign':'left'}),
    dcc.Slider(.1, 1, .1,
               value=.05,
               id='slider1'
    ),
    html.P(children='Attenuation Coefficient (a)', style={'textAlign':'left'}),
    dcc.Slider(.1, 1, .1,
               value=.05,
               id='slider2'
    ),
    html.P(id='finesse_allpass'),
    dcc.Graph(id='graph-content1'),
    html.P(children='Coupling Coefficient (r1)', style={'textAlign':'left'}),
    dcc.Slider(.1, 1, .1,
               value=.05,
               id='slider3'
    ),
    html.P(children='Coupling Coefficient (r2)', style={'textAlign':'left'}),
    dcc.Slider(.1, 1, .1,
               value=.05,
               id='slider4'
    ),
    html.P(children='Attenuation Coefficient (a)', style={'textAlign':'left'}),
    dcc.Slider(.1, 1, .1,
               value=.05,
               id='slider5'
    ),
    html.P(id='finesse_add_drop'),
    dcc.Graph(id='graph-content2')
])

@callback(
    Output(component_id='finesse_allpass', component_property='children'),
    Input('slider1', 'value'),
    Input('slider2', 'value')
)
def update_output_div(r_value, a_value):
    return f'Finesse: {round(finesse_allpass(r_value, a_value), 2)}'

@callback(
    Output('graph-content1', 'figure'),
    Input('slider1', 'value'),
    Input('slider2', 'value')
)
def update_graph(r_value, a_value):
    fig = make_subplots(rows=1, cols=2)
    fig.update_layout(title_text="All Pass Ring Resonator - Phase Delay and Transmission Intensity", template="plotly_white")
    fig.add_trace(go.Line(x=detuning, y = phase_shift(r_value, a_value, detuning)), row=1, col=1)
    fig.add_trace(go.Line(x=detuning_long, y = transmission_intensity(r_value, a_value, detuning_long)), row=1, col=2)
    fig.update_xaxes(row = 1, col = 1, nticks=2, tickvals=[-pi, pi], ticktext=['-π', 'π'])
    fig.update_yaxes(row = 1, col = 1, title_text = "Effective Phase Delay", nticks=2, tickvals=[0, 2*pi], ticktext=['0', '2π'])
    fig.update_xaxes(row = 1, col = 2, nticks=2, tickvals=[-pi, pi], ticktext=['-π', '3π'])
    fig.update_yaxes(row = 1, col = 2, nticks=2, title_text = "Transmission Delay", tickvals=[0, 2*pi], ticktext=['0', '2π'])
    fig.update_xaxes(title_text = "Detuning Angle")
    return fig

@callback(
    Output(component_id='finesse_add_drop', component_property='children'),
    Input('slider3', 'value'),
    Input('slider4', 'value'),
    Input('slider5', 'value')
)
def update_output_div(r1_value, r2_value, a_value):
    return f'Finesse: {round(finesse_add_drop(r1_value, r2_value, a_value), 2)}'

@callback(
    Output('graph-content2', 'figure'),
    Input('slider3', 'value'),
    Input('slider4', 'value'),
    Input('slider5', 'value')
)

def update_graph2(r1_value, r2_value, a_value):
    fig2 = go.Figure()
    fig2.update_layout(title_text="Add Drop Ring Resonator - Transmission Pass and Drop", template="plotly_white")
    fig2.add_trace(go.Line(x=detuning_long, y = transmission_pass(r1_value, r2_value, a_value, detuning_long)))
    fig2.add_trace(go.Line(x=detuning_long, y = transmission_drop(r1_value, r2_value, a_value, detuning_long)))
    fig2.update_xaxes(nticks=2, tickvals=[-pi, pi], ticktext=['-π', '3π'])
    fig2.update_yaxes(nticks=2, tickvals=[0, 2*pi], ticktext=['0', '2π'])
    return fig2

if __name__ == '__main__':
    app.run(debug=True)